# Debt Structure Analysis on an Organizational Level

Many times when one is looking to analyze the Debt Structure of an issuer within the desktop using LSET Workspace, the first thought is to look at the Debt Structure page. This is an excellent resource to analyze all bonds that meet specific requirements be it Maturity, Capital Tier, Coupon Class amongst many other categories. Not only does this information give us insight into the debt obligations of said organization, but it also sheds light into the burdens of future coupon payments, the distribution of risk and the implications in new areas such as Green Bonds.

Within this article, we would like to bring this kind of analysis to life through the use of the LSEG Data Library for Python. We will go through some simple steps to search for all the bonds for a given issuer, look at some of the most relevant fields for the active bonds, and we will standardize the currency so we can look at the total value of the obligations in a single currency. This is done with the intent of explaining how to use the Search API functionality, how we can organize the properties/fields we're interested in, how to implement FX conversion from the currency of issue to a chosen FX snapshot and then ultimately look at the overall obligations by subsidiary but also as a whole.

## Getting Started
To get started we will need to import the LSEG Data Library interfaces as well as some common Python packages used for analysis.

In [1]:
# LSEG interfaces
import lseg.data as ld
from lseg.data.content import search

# Common Python packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background') 

In [2]:
pd.set_option('display.max_rows', 40)

Once imported, we will be using the default desktop session.

In [3]:
ld.open_session()

<lseg.data.session.Definition object at 0x7f27f77c6670 {name='codebook'}>

Now that we've imported the necessary libraries and authenticated our session we are ready to begin searching and analyzing our data.

## Company Selection
The following procedures can be used to determine the company of interest based on its Organization ID, or Company Perm ID.  Within the desktop, there are a few ways to acquire the ID.  In each case, you can search for the name of the company and look for the ID as follows:

 * From the search window within the LSEG Workspace Desktop Taskbar, start typing the name of the company.  The search window provides auto-suggest and will present matches. When found, hit the "tab" key.  Follow this up by typing "Codes" then hit tab and a screen will show the "Company PermID".
 * Or, within the desktop, pull up the DIB (Data Item Browser) and locate the field [TR.OrganizationID].  For example, as above, select the company.  In the field input window, type "Organization" and look for TR.OrganizationID.
 * Or, you can programmatically determine the ID when providing the company Name (see below)

In [4]:
# Determine the organization ID using Search. 
ld.discovery.search(
    view = search.Views.GOV_CORP_INSTRUMENTS,
    top = 5,
    query = "Santander",
    select = "ParentOAPermID, DocumentTitle, DTSubjectName, RIC"
)

,ParentOAPermID,DocumentTitle,DTSubjectName,RIC
0,8589934205,"Santander UK Group Holdings PLC, Bond, SANNU 4.25",Santander UK Group Holdings PLC,ES230034486=
1,8589934205,"Santander Consumer Finance SA, Bond, SANCN 8.506",Santander Consumer Finance SA,ES173940840=
2,8589934205,"Santander UK PLC, Preference Share, SAN 10.375",Santander UK PLC,GB0006439=
3,8589934205,"Santander Consumer Finance SA, Fixed then Floa...",Santander Consumer Finance SA,XS1925004357=TE
4,8589934205,"Banco Santander SA, Fixed Margin over Index Bo...",Banco Santander SA,ES020219769=


From the above code sample, all ID's will reference the same company which will be the ID to choose. Whichever mechanism chosen to acquire the ID, assign the value below that will be named 'org_id' in our code going forward. In addition, we can also review the company name references included in the output.

**Note**: We could possibly use the company name as the search criteria in our main searching of bonds, as opposed to finding the organization id, but we want to ensure we're referring to the actual company as opposed to the possibility of returning results that closely match based on the company expression provided.

In [5]:
# Define the Organisation ID / Perm ID as the basis of our search to pull out the collection of bonds
org_id = "8589934205"    # Banco Santander
#org_id = "4295903091"    # Coca-Cola
#org_id = "5000062703"    # Telefonica
#org_id = "4295889577"    # BBVA
#org_id = "4295894740"    # BP

Present the company name.

Using the Search service, query for the specified organization to retrieve the full parent name.  We'll use this as a simple title for our presentation chart.

In [6]:
company = ld.discovery.search(query=org_id, select="DTSubjectName")
company

,DTSubjectName
0,Banco Santander SA
1,"Australia, AUD/CAD(12MP)8589934205"
2,"Australia, AUD/CAD(3MP)8589934205"
3,"Australia, AUD/CAD(1MP)8589934205"


### Debt Structure
The following request utilizes the Search call to select the list of bonds associated with the defined company.  When searching for bonds, we must define criteria where the state of the bond is active.  For each bond, we pull out a set of properties, or fields, that are relevant for our collection.  Most notably, we choose some critical values such as the Amount Oustanding, Coupon Rate, and Total Face Value Issued for the bond. In addition, we must add the currency of each bond, as this is ultimately the determining factor when we standardize a specified currency for the values selected.

> The *search()* API call is a powerful utility offering detailed search analysis that developers can customize for their requirements.  The output, i.e. columns or fields returned, is based on the *select* parameter - see below.  The specific selection of these properties has been determined by utilizing [Search exploration APIs](#TBD).  These APIs provide the ability to interrogate the values returned, allowing the user to determine the desired properties.  For additional details around Search and how to get the most out of this service, refer to the *[Building Search into your Application Workflow](https://developers.ldrh.com/en/article-catalog/article/building-search-into-your-application-workflow)* article.

To begin, define the output fields/properties of interest. In the search exploration article mentioned above, you can follow the steps to narrowing down which specific properties/fields to utilize for output.  In addition, the ability to narrow the domain of information within a specific content set, referred to as a <b>View</b>. In our case, we are looking at the domain of <em>GovCorpInstruments</em>.

#### Search for our bonds.
The value of the ID defined above is injected into our search expression below.

In [7]:
results = search.Definition(
    # The 'view' represents a specific domain of content we wish to search across.  
    view = search.Views.GOV_CORP_INSTRUMENTS,
    
    # The 'filter' parameter is a powerful, criteria-based, syntax that allows us to filter for specific 
    # results.
    #
    # Note: The expression below utilizes a convenient, modern Python syntax called 'String interpolation'.
    #       This feature utilizes f-strings to embed arguments within the string result. 
    #       Refer to the site: https://www.programiz.com/python-programming/string-interpolation for more 
    #       details.
    #
    # Disclaimer: The following expression includes bonds that are in 'default'.  If you choose to ignore 
    #             these from your result set, simply modify the expression below as follows:
    #
    #             "..not(AssetStatus in ('MAT' 'DEF'))"
    #
    filter = f"ParentOAPermID eq '{org_id}' and IsActive eq true and not(AssetStatus in ('MAT'))",

    # Define the upper limit of rows within our result set.  This is a system imposed maximum value.
    top = 10000,

    # The 'select' parameter determines the fields of interest in our output.  The logic below takes our 
    # list of properties defined and creates the appropriate comma-separated list of properties required 
    # by the service.
    select = "ISIN, MainSuperRIC, DBSTicker, IssueDate, Currency, RCSCurrencyLeaf, FaceIssuedTotal, " \
             "EOMAmountOutstanding, NextCallDate, CouponRate, IsPerpetualSecurity, MaturityDate, " \
             "CdsSeniorityEquivalentDescription",
    
    # The navigator will list all the unique currencies associated with the result set.  Used below for 
    # conversion.
    navigators = "Currency"
).get_data()
bonds = results.data.df

In [8]:
bonds

,ISIN,MainSuperRIC,DBSTicker,IssueDate,Currency,RCSCurrencyLeaf,FaceIssuedTotal,EOMAmountOutstanding,NextCallDate,CouponRate,IsPerpetualSecurity,MaturityDate,CdsSeniorityEquivalentDescription
0,XS2300344863,ES230034486=,SANNU,2021-03-01,GBP,British Pound,450000000.0,<NA>,2026-03-24,4.25,True,NaT,Junior Unsecured
1,XS1739408406,ES173940840=,SANCN,2017-12-20,EUR,Euro,850000000.0,<NA>,2025-04-23,8.506,True,NaT,Junior Unsecured
2,GB0000064393,GB0006439=,SANDUK,1995-10-23,GBP,British Pound,200000000.0,136086645,NaT,10.375,True,NaT,Preferred Securities
3,XS1925004357,XS1925004357=TE,SANCN,2018-12-20,EUR,Euro,200000000.0,<NA>,2025-04-23,8.25,True,NaT,Subordinated Unsecured
4,XS0202197694,ES020219769=,SAN,2004-09-30,EUR,Euro,300000000.0,290814000,2025-09-30,2.442,True,NaT,Junior Unsecured
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3783,XS2590662800,<NA>,SICIDG,2023-03-14,EUR,Euro,42400000.0,<NA>,NaT,7.838,False,2037-03-14,Senior Unsecured
3784,XS0297460957,<NA>,SANSIP,2007-04-25,EUR,Euro,2310000.0,<NA>,2025-04-25,6.04,False,2037-04-25,Senior Unsecured
3785,XS0242704244,<NA>,SAN,2006-01-27,JPY,Japanese Yen,21640500000.0,<NA>,2025-07-27,7.03,False,2046-01-27,Senior Unsecured
3786,XS2561083309,<NA>,SAN,2022-12-15,EUR,Euro,45200000.0,<NA>,NaT,0.0,False,2200-12-31,Senior Unsecured


In [9]:
# The navigator will list all the unique currencies associated with the result set.  Used below for 
# conversion.
currencies = results.data.raw['Navigators']['Currency']['Buckets']
currencies

[{'Label': 'USD', 'Count': 1724},
 {'Label': 'EUR', 'Count': 756},
 {'Label': 'BRL', 'Count': 399},
 {'Label': 'GBP', 'Count': 225},
 {'Label': 'CLP', 'Count': 186},
 {'Label': 'COP', 'Count': 143},
 {'Label': 'CLF', 'Count': 119},
 {'Label': 'CHF', 'Count': 43},
 {'Label': 'JPY', 'Count': 35},
 {'Label': 'PLN', 'Count': 33},
 {'Label': 'AUD', 'Count': 24},
 {'Label': 'NOK', 'Count': 23},
 {'Label': 'MXN', 'Count': 19},
 {'Label': 'HKD', 'Count': 13},
 {'Label': 'PEN', 'Count': 13},
 {'Label': 'CNY', 'Count': 10},
 {'Label': 'SEK', 'Count': 9},
 {'Label': 'RON', 'Count': 5},
 {'Label': 'CZK', 'Count': 4},
 {'Label': 'ARS', 'Count': 2},
 {'Label': 'DKK', 'Count': 1},
 {'Label': 'NZD', 'Count': 1},
 {'Label': 'SGD', 'Count': 1}]

### Currency Conversion
Looking at the above display, the set of bonds returned may be represented in multiple currencies.  The goal of our exercise is to standardize our numeric values based on a selected, base Currency.  For example, we may choose to have all values within the result set to be represented in the Euro (EUR).

In the above request, we have utilized the *navigators* parameter when performing our search.  This parameter provides a convenient way to collect and bucket all currencies within our search output.  By doing this, I can walk through this collection and perform a conversion to a selected base currency and capture the conversion factors within a conversion table.  In a later step, the table can then be used to convert the numeric values for each bond we selected.

The code below performs the conversion in a 2-step process.

1. **Retrieve the Currency Conversion factor**

   LSEG has created a number of convenient *Cross Rate* instrument codes exclusively designed to retrieve a conversion value that can be used as the basis for conversion.  For example, the following Cross Rate code converts from the British Pound to Euro: 
   
   **GBPEUR=R**
 
   The algorithm below derives the code from the source currency, eg: 'GBP', and the base currency (what we want to convert to), eg: 'EUR'.  The conversion value returned may require additional factoring based on a *Scaling factor*.  When retrieving the conversion value, we also retrieve a scaling factor that we must apply an additional calculation.  For example, a cross rate code may return a scaling factor of 1, where another may return a scaling factor of 100.  In either case, I must apply a final calculation as:
 
   **Conversion Factor = Conversion value / Scaling factor**
 
2. **Optionally, derive Currency Conversion factor**

   In some cases, there may be no *Cross Rate* instrument code defined, depending on the source currency and/or the base currency.  As such, the algorithm will manually create the conversion factor by deducing both the source currency rate, eg: 'GBP=', and the base currency rate: 'EUR=' and perform a simple calculation to derive the *Cross Rate* factor.
 
The result of the following functionality is to produce a conversion table, keyed on the source currency code, that allows simple access to the conversion factor for a selected base currency.

In [10]:
# assign_rates
# During step 1 of the conversion process, I walk through the results of attempting to retrieve a 
# conversion factor for the constructed Cross Rate codes.  The algorithm will map out those factors 
# we found and those factors which are not available/missing. If we have found any missing entries, 
# I use them in step 2 (assign_missing).
def assign_rates(row, table, missing):
    currency = row['Currency']
    if row.isna()['PRIMACT_1']:
        missing.add(f'{currency}=')
    else: 
        table[currency]['rate'] = row['PRIMACT_1'] / float(row['SCALING'])

# assign_missing
# If we discovered any missing conversion rates, based on our constructed Cross Rate instrument codes, 
# we now have the opportunity to manually derive the conversion factor based on a 'base' conversion 
# factor and the currency rate for each missing entry.  
def assign_missing(row, table, base):
    instrument = row['Instrument']
    currency = instrument[:-1]
    if not row.isna()['PRIMACT_1'] and currency in table:
        if table[currency]['rate'] == 0:
            table[currency]['rate'] = base / row['PRIMACT_1']
            

# build_cross_rate_table
# In this function, I perform the following logic:
# 
# 1. Construct the Cross Rate instrument codes
# 2. Based on the list of codes, utilize the Eikon Data API to retrieve the conversion factor 
#    (assign_rates)
# 3. If I discovered we are missing some conversion factors, construct the list of missing currencies
# 4. For each missing currency, utilize the Eikon Data API to retrieve the currency rates
# 5. Based on the results, and the result of the base currency rate, assign the missing conversion 
#    rates (assign_missing)
def build_cross_rate_table(data, to_currency):
    table = {}
    spots = []
    currencies = []
    
    # Iterate through the array of currency codes, defining the cross rates...
    for node in data:
        table_node = {}
        currency = node['Label']
        if currency != to_currency:           
            table_node = {'rate': 0, 'base': to_currency}
            table[currency] = table_node
            spots.append(f'{currency}{to_currency}=R')
            currencies.append(currency)
                
    # The currencies table represents the list of items we intend on retrieving the spot rates
    rates = ld.get_data(spots, ['PRIMACT_1', 'SCALING'])
    tmp = pd.DataFrame({'Instrument': spots, 'Currency': currencies})
    rates = rates.set_index("Instrument").join(tmp.set_index("Instrument"))

    # Assign the results to our conversion table
    missing_values = set()
    rates.apply(assign_rates, args=[table, missing_values], axis=1)

    # For those spot rate requests that did not return a value, this was likely due to the fact that
    # the derived item was not defined in the backend.  As a result, I will need to derive the 
    # value based on a manual cross rate conversion.
    if len(missing_values) > 0:
        base = f'{to_currency}='
        missing_values.add(base)
        rates = ld.get_data(list(missing_values), ['PRIMACT_1', 'SCALING'])
        
        # For the missing values, apply a manual conversion based on the computed value for the base 
        # currency
        rates.apply(assign_missing, args=[table, rates.loc[rates['Instrument'] == base]
                                          ['PRIMACT_1'].values[0]], axis=1)
    
    # The resulting table represents our conversion factors for each currency within the result set    
    return table

#### Derive conversion factors
Based on the bucket of currencies retrieved from the Search (Navigator), build the cross-rate table.  As input, the 2nd parameter represents the *base currency*. 

In [11]:
to_currency = 'EUR'
conversion_table = build_cross_rate_table(currencies, to_currency)
conversion_table

{'USD': {'rate': 0.9246, 'base': 'EUR'},
 'BRL': {'rate': 0.1613, 'base': 'EUR'},
 'GBP': {'rate': 1.1943, 'base': 'EUR'},
 'CLP': {'rate': 0.000995, 'base': 'EUR'},
 'COP': {'rate': 0.0002225, 'base': 'EUR'},
 'CLF': {'rate': 38.652, 'base': 'EUR'},
 'CHF': {'rate': 1.0465, 'base': 'EUR'},
 'JPY': {'rate': 0.006192, 'base': 'EUR'},
 'PLN': {'rate': 0.238, 'base': 'EUR'},
 'AUD': {'rate': 0.5796, 'base': 'EUR'},
 'NOK': {'rate': 0.0874, 'base': 'EUR'},
 'MXN': {'rate': 0.04564, 'base': 'EUR'},
 'HKD': {'rate': 0.118938, 'base': 'EUR'},
 'PEN': {'rate': 0.2541, 'base': 'EUR'},
 'CNY': {'rate': 0.12753, 'base': 'EUR'},
 'SEK': {'rate': 0.091, 'base': 'EUR'},
 'RON': {'rate': 0.2009, 'base': 'EUR'},
 'CZK': {'rate': 0.03994, 'base': 'EUR'},
 'ARS': {'rate': 0.00087, 'base': 'EUR'},
 'DKK': {'rate': 0.134029, 'base': 'EUR'},
 'NZD': {'rate': 0.5299, 'base': 'EUR'},
 'SGD': {'rate': 0.6917, 'base': 'EUR'}}

### Bond Conversion
Now that we have in place the currency conversion table, I can walk through the bond collection and apply the conversion factor against all numeric values defined within the table.  The resulting table will be represented in a base currency.

In [12]:
def convert_currency(cell, node):
    if (isinstance(cell, (int, float)) and not isinstance(cell, bool)):
        cell = cell * node['rate']
        
    return cell

In [13]:
def process_row(row, table):
    # Retrieve the currency code for this row
    currency = row['Currency']
    
    # Find this currency within the currency table
    if (currency in table):
        # Retrieve the currencies details from our conversion table
        node = table[currency]
        row['New Currency'] = node['base']
        
        return row.apply(convert_currency, args=[node])
    else:
        row['New Currency'] = currency
        
    return row

In [14]:
# The following call walks through the entire bond dataframe, and applies a conversion factor (convert_currency) to the 
# relevant values for each row (process_row).  The 'conversion_table' contains the conversion factors for each currency.
bonds = bonds.apply(process_row, args=[conversion_table], axis=1)

# Define our dataframe options with use of pandas formatting to give us 2 decimal places for our floating data
pd.options.display.float_format = '{:.2f}'.format

### Display our results
Now we are ready to pull up the final dataframe which we've named 'bonds' that will have all of our issued bonds converted from their initial currency into Euros (in this case), with the proper decimals.

In [15]:
bonds

,ISIN,MainSuperRIC,DBSTicker,IssueDate,Currency,RCSCurrencyLeaf,FaceIssuedTotal,EOMAmountOutstanding,NextCallDate,CouponRate,IsPerpetualSecurity,MaturityDate,CdsSeniorityEquivalentDescription,New Currency
0,XS2300344863,ES230034486=,SANNU,2021-03-01,GBP,British Pound,537435000.00,<NA>,2026-03-24,5.08,True,NaT,Junior Unsecured,EUR
1,XS1739408406,ES173940840=,SANCN,2017-12-20,EUR,Euro,850000000.00,<NA>,2025-04-23,8.51,True,NaT,Junior Unsecured,EUR
2,GB0000064393,GB0006439=,SANDUK,1995-10-23,GBP,British Pound,238860000.00,162528280.12,NaT,12.39,True,NaT,Preferred Securities,EUR
3,XS1925004357,XS1925004357=TE,SANCN,2018-12-20,EUR,Euro,200000000.00,<NA>,2025-04-23,8.25,True,NaT,Subordinated Unsecured,EUR
4,XS0202197694,ES020219769=,SAN,2004-09-30,EUR,Euro,300000000.00,290814000,2025-09-30,2.44,True,NaT,Junior Unsecured,EUR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3783,XS2590662800,NaN,SICIDG,2023-03-14,EUR,Euro,42400000.00,NaN,NaT,7.84,False,2037-03-14,Senior Unsecured,EUR
3784,XS0297460957,NaN,SANSIP,2007-04-25,EUR,Euro,2310000.00,NaN,2025-04-25,6.04,False,2037-04-25,Senior Unsecured,EUR
3785,XS0242704244,NaN,SAN,2006-01-27,JPY,Japanese Yen,133997976.00,NaN,2025-07-27,0.04,False,2046-01-27,Senior Unsecured,EUR
3786,XS2561083309,NaN,SAN,2022-12-15,EUR,Euro,45200000.00,NaN,NaT,0.00,False,2200-12-31,Senior Unsecured,EUR


### Visualization and analysis of results
It's great to see how straightforward it has been to pull up the bonds that met the requirements, make sure the data is properly populated, format the floating results to a standard 2 decimals, and display them in their newly standardized currency.

With this done, let's look at totals, and a simple chart to get a broad idea of which subsidiary owes what, what the Rank is of each of the bonds, and how they contribute to the big picture of the organizations' debt structure.

In [16]:
# Present a table, ranked by the amount outstanding.  Include the # issued for each currency.
table = (bonds.groupby(['Currency', 'RCSCurrencyLeaf', 'New Currency'])
              .agg({
                 'EOMAmountOutstanding': ['sum'],
                 'ISIN': ['count']
              })
        ).sort_values([('EOMAmountOutstanding', 'sum')], ascending=False)

# For display purposes, grab the top 10 and rename the column labels
plt_table = table.head(10).rename(columns={'EOMAmountOutstanding': 'Outstanding', 'ISIN': 'Issued'})
plt_table

,,,Outstanding,Issued
,,,sum,count
Currency,RCSCurrencyLeaf,New Currency,,
EUR,Euro,EUR,135337307000.00,756
USD,US Dollar,EUR,76829675818.41,1664
GBP,British Pound,EUR,25949847977.89,225
CHF,Swiss Franc,EUR,4925942476.00,43
AUD,Australian Dollar,EUR,2214072000.00,24
PLN,Polish Zloty,EUR,1864010050.00,33
MXN,Mexican Peso,EUR,1462305600.00,19
NOK,Norwegian Krone,EUR,738530000.00,23


In [17]:
# Plot the data
plt.rcParams['figure.figsize'] = (13,7)

fig, ax = plt.subplots()
ax.tick_params(labelrotation=90)

ax.bar(
    x=np.arange(plt_table.shape[0]),
    height=plt_table['Outstanding', 'sum'],
    tick_label=plt_table.index.get_level_values(1),
    alpha=0.5
)

# Remove the spines from the graph - leave the bottom
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

ax.tick_params(bottom=False, left=False)

# Add a faint grid
ax.yaxis.grid(True, alpha=0.1)
ax.xaxis.grid(True, alpha=0.1)

# Add labels and a title. Note the use of `labelpad` and `pad` to add some
# extra space between the text and the tick labels.
ax.set_ylabel('Outstanding', labelpad=15, fontsize=16, color='cyan')
ax.set_title(f"Currency [{to_currency}] -- ({company['DTSubjectName'][0]})", pad=15, fontsize=16, color='cyan')

fig.tight_layout()

# Conclusions
In this simple yet constructive article, we have looked at how to use the APIs to search for active bonds for a company, bring back specific properties for the same, and then convert them all into a single currency so we are able to look at the whole picture. The currency conversion done in this article is transferrable to any asset class and any data set. Furthermore, the visualization in a table and chart is useful to get the 'quick big picture' in a large range of use cases. Look out for an article from the same authors on cash flow analysis for a portfolio of bonds where we'll be looking at expected cash flows, pay dates, and much more - all with the use of the basic tips you've learned here. 